## 1. Products sold online but never sold in stores
#### Functional Specification
* Use the Sales.SalesOrderHeader and Sales.SalesOrderDetail tables
* In the first query, select all ProductIDs from online orders (OnlineOrderFlag = 1).
* In the second query, select all ProductIDs from store (in-person) orders (OnlineOrderFlag = 0).
* Use the EXCEPT operator to return only those products that appear in online sales but not in store sales.
* DISTINCT ensures each product appears only once.

In [ ]:
USE AdventureWorks2022;

SELECT DISTINCT ProductID
FROM Sales.SalesOrderHeader AS h
JOIN Sales.SalesOrderDetail AS d ON h.SalesOrderID = d.SalesOrderID
WHERE h.OnlineOrderFlag = 1
EXCEPT
SELECT DISTINCT ProductID
FROM Sales.SalesOrderHeader AS h
JOIN Sales.SalesOrderDetail AS d ON h.SalesOrderID = d.SalesOrderID
WHERE h.OnlineOrderFlag = 0;

## 2. Customers who purchased in 2023 but not 2024
#### Functional Specification
* Retrieve all unique CustomerIDs from Sales.SalesOrderHeader where the order year is 2023.
* Retrieve all unique CustomerIDs where the order year is 2024.
* Use the EXCEPT operator to find customers who placed orders in 2023 but did not place any in 2024.
* This identifies customers who became inactive after 2023.

In [ ]:
USE AdventureWorks2022;

SELECT DISTINCT CustomerID
FROM Sales.SalesOrderHeader
WHERE YEAR(OrderDate) = 2023
EXCEPT
SELECT DISTINCT CustomerID
FROM Sales.SalesOrderHeader
WHERE YEAR(OrderDate) = 2024;

## 3. Employees who have changed departments
#### Functional Specification
* Query the HumanResources.EmployeeDepartmentHistory table.
* Group the records by BusinessEntityID to aggregate department history per employee.
* Use HAVING COUNT(DISTINCT DepartmentID) > 1 to find employees associated with more than one department.
* The result lists employees who have transferred between departments.

In [ ]:
USE AdventureWorks2022;

SELECT BusinessEntityID
FROM HumanResources.EmployeeDepartmentHistory
GROUP BY BusinessEntityID
HAVING COUNT(DISTINCT DepartmentID) > 1;

## 4. Top 10 products sold online + Top 10 sold in stores
#### Functional Specification
* Perform two separate aggregations of sales totals (SUM(LineTotal)) from the joined tables Sales.SalesOrderHeader and Sales.SalesOrderDetail.
* In the first query, filter where OnlineOrderFlag = 1 (online sales) and select the top 10 based on total sales.
* In the second query, filter where OnlineOrderFlag = 0 (in-store sales) and select the top 10.
* Use the UNION operator to combine both lists into one result set of top-performing products across both sales channels.


In [ ]:
USE AdventureWorks2022;

SELECT TOP 10 d.ProductID, SUM(d.LineTotal) AS TotalSales
FROM Sales.SalesOrderHeader AS h
JOIN Sales.SalesOrderDetail AS d ON h.SalesOrderID = d.SalesOrderID
WHERE h.OnlineOrderFlag = 1
GROUP BY d.ProductID
UNION
SELECT TOP 10 d.ProductID, SUM(d.LineTotal) AS TotalSales
FROM Sales.SalesOrderHeader AS h
JOIN Sales.SalesOrderDetail AS d ON h.SalesOrderID = d.SalesOrderID
WHERE h.OnlineOrderFlag = 0
GROUP BY d.ProductID;


## 5. Customers living in either California or New York
#### Functional Specification
* Join the Sales.Customer, Person.Address, and Person.StateProvince tables to retrieve each customer’s city and state.
* In the first query, filter for customers located in California.
* In the second query, filter for those located in New York.
* Use UNION to merge both sets while automatically removing duplicates.
* The result lists all customers who reside in either of the two states.


In [ ]:
USE AdventureWorks2022;

SELECT DISTINCT c.CustomerID, a.City, sp.Name AS State
FROM Sales.Customer AS c
JOIN Person.Address AS a ON c.CustomerID = a.AddressID
JOIN Person.StateProvince AS sp ON a.StateProvinceID = sp.StateProvinceID
WHERE sp.Name = 'California'
UNION
SELECT DISTINCT c.CustomerID, a.City, sp.Name AS State
FROM Sales.Customer AS c
JOIN Person.Address AS a ON c.CustomerID = a.AddressID
JOIN Person.StateProvince AS sp ON a.StateProvinceID = sp.StateProvinceID
WHERE sp.Name = 'New York';

## 6. Products common to two promotions
#### Functional Specification
* Query the Sales.SpecialOfferProduct table twice:
    * Once to select all ProductIDs under SpecialOfferID = 1.
    * Again to select all ProductIDs under SpecialOfferID = 2.
* Use INTERSECT to find products that appear in both promotions.
* This identifies overlapping promotional items.


In [ ]:
USE AdventureWorks2022;

SELECT ProductID
FROM Sales.SpecialOfferProduct
WHERE SpecialOfferID = 1
INTERSECT
SELECT ProductID
FROM Sales.SpecialOfferProduct
WHERE SpecialOfferID = 2;

## 7. Customers with open orders and customers with past-due invoices
#### Functional Specification
* Use the Sales.SalesOrderHeader table to identify two customer groups:
    * Those with open orders (Status = 1).
    * Those with past-due or complete orders (Status = 5).
* Combine both customer lists using UNION to include all unique customers who currently have active or past-due transactions.
* This helps monitor customers with ongoing or unsettled orders


In [ ]:
USE AdventureWorks2022;

SELECT DISTINCT CustomerID
FROM Sales.SalesOrderHeader
WHERE Status = 1   
UNION
SELECT DISTINCT CustomerID
FROM Sales.SalesOrderHeader
WHERE Status = 5; 

## 

## 8. Suppliers that sell both high-priced and low-priced products
#### Functional Specification
* Query Purchasing.ProductVendor joined with Production.Product to access suppliers and their products’ prices.
* In one subquery, select suppliers selling items priced above $50.
* In another, select suppliers selling items priced below $200.
* Use INTERSECT to find suppliers appearing in both sets.
* The result identifies vendors that offer both premium and budget product ranges.

In [ ]:
USE AdventureWorks2022;

SELECT v.BusinessEntityID, v.Name
FROM Purchasing.Vendor AS v
WHERE v.BusinessEntityID IN (
    SELECT DISTINCT pv.BusinessEntityID
    FROM Purchasing.ProductVendor AS pv
    JOIN Production.Product AS p ON pv.ProductID = p.ProductID
    WHERE p.ListPrice > 50
    INTERSECT
    SELECT DISTINCT pv.BusinessEntityID
    FROM Purchasing.ProductVendor AS pv
    JOIN Production.Product AS p ON pv.ProductID = p.ProductID
    WHERE p.ListPrice < 200
);


## 9. Distinct job titles across current and past employee records
#### Functional Specification
* Join HumanResources.Employee with HumanResources.EmployeeDepartmentHistory on BusinessEntityID.
* Select distinct JobTitles to avoid duplicates.
* This retrieves all unique job roles associated with employees who appear in the department history table.
* It’s useful for identifying all active and historical job titles in the organization.


In [ ]:
USE AdventureWorks2022;

SELECT DISTINCT e.JobTitle
FROM HumanResources.Employee AS e
JOIN HumanResources.EmployeeDepartmentHistory AS edh
    ON e.BusinessEntityID = edh.BusinessEntityID;

## 10. All distinct people appearing as customers, employees, or vendors
#### Functional Specification
* Select identifiers from three different tables, tagging each record with its role:
    * From HumanResources.Employee (as 'Employee')
    * From Sales.Customer (as 'Customer')
    * From Purchasing.Vendor (as 'Vendor')
* Combine all three sets using UNION to create a single, deduplicated list of all business entities.
* This shows every individual or organization participating in any capacity within the company ecosystem.


In [ ]:
USE AdventureWorks2022;

SELECT BusinessEntityID, 'Employee' AS Role
FROM HumanResources.Employee
UNION
SELECT PersonID, 'Customer' AS Role
FROM Sales.Customer
UNION
SELECT BusinessEntityID, 'Vendor' AS Role
FROM Purchasing.Vendor;